In [2]:
import numpy as np
import pandas as pd
import itertools as it
import math as mt

# Ejercicio II.1

### Se ha clasificado el peso de los huevos, $Y$, de una especie de pez en función del peso de la madre, $X$, como refleja la tabla adjunta, y se pide estimar una serie de características de la distribución.

$X$/$Y$    | [25, 27) | [27, 29) | [29, 31) | [31, 33)
---------- | -------- | -------- | -------- | --------
**[500, 550)** | 15       | 11       | 18       | 0
**[550, 600)** | 12       | 14       | 0        | 12
**[600, 650)** | 0        | 3        | 7        | 18

En primer lugar, se construye con código la tabla bidimensional del enunciado; se definen sendas listas con los intervalos en que se han agrupado ambas variables, $X$ e $Y$, y se indican las **frecuencias absolutas** $n_{ij}$ de cada **par de valores** $(x_{i}, y_{j})$:

In [3]:
y = ["[25, 27)", "[27, 29)", "[29, 31)", "[31, 33)"] * 3
y = np.repeat(y, [15, 11, 18, 0, 12, 14, 0, 12, 0, 3, 7, 18], axis = 0)

x = ["[500, 550)"] * 4 + ["[550, 600)"] * 4 + ["[600, 650)"] * 4 
x = np.repeat(x, [15, 11, 18, 0, 12, 14, 0, 12, 0, 3, 7, 18], axis=0)

tabla_2d = pd.crosstab(index = x, columns = y, rownames = ['X'], colnames=['Y'])
tabla_2d

Y,"[25, 27)","[27, 29)","[29, 31)","[31, 33)"
X,,,,
"[500, 550)",15,11,18,0
"[550, 600)",12,14,0,12
"[600, 650)",0,3,7,18


### a) *Distribución del peso del huevo*

Equivale a calcular la **distribución marginal** de $Y$; es decir, sus **frecuencias absolutas marginales** $n_{.j}$ :

In [4]:
n·j = tabla_2d.sum()
n·j

Y
[25, 27)    27
[27, 29)    28
[29, 31)    25
[31, 33)    30
dtype: int64

Siendo la distribución marginal de $X$:

In [5]:
ni· = tabla_2d.transpose().sum()
ni·

X
[500, 550)    44
[550, 600)    38
[600, 650)    28
dtype: int64

El **número total de observaciones**, $n$ o $n_{..}$:

In [6]:
n = n·j.sum() #ni·.sum() daría el mismo resultado
n

110

En conjunto:

In [7]:
tabla_2d_m = pd.crosstab(index = x, columns = y, margins = True)

tabla_2d_m.index = ["[500, 550)", "[550, 600)", "[600, 650)", "Huevo"]
tabla_2d_m.columns = ["[25, 27)", "[27, 29)", "[29, 31)", "[31, 33)", "Madre"]

tabla_2d_m

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",15,11,18,0,44
"[550, 600)",12,14,0,12,38
"[600, 650)",0,3,7,18,28
Huevo,27,28,25,30,110


### b) *Distribución del peso de la madre cuando el huevo tiene el suyo comprendido entre 25 y 27*

Se trata de la **distribución condicionada** de $X$ a $Y$ cuando ésta toma valores únicamente en el intervalo [25, 27); es decir, las **frecuencias relativas condicionadas** $f_{i|1}$:

In [8]:
fi_1 = tabla_2d.transpose().iloc[0]/tabla_2d_m.iloc[3, 0]

fi_1r = round(fi_1, 2)
fi_1r

X
[500, 550)    0.56
[550, 600)    0.44
[600, 650)    0.00
Name: [25, 27), dtype: float64

Si se calcula para todos los intervalos de peso de los huevos, $f_{i|j}$:

In [9]:
fi_j = tabla_2d_m.div(tabla_2d_m.loc["Huevo"], axis = 1)

fi_jr = round(fi_j, 2)
fi_jr

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",0.56,0.39,0.72,0.0,0.40
"[550, 600)",0.44,0.50,0.00,0.4,0.35
"[600, 650)",0.00,0.11,0.28,0.6,0.25
Huevo,1.00,1.00,1.00,1.0,1.00


Y si se quiere calcular la distribución del peso de los huevos condicionada al peso de la madre, $f_{j|i}$:

In [10]:
fj_i = tabla_2d_m.div(tabla_2d_m["Madre"], axis = 0) #No hay que usar "loc" en este caso

fj_ir = round(fj_i, 2)
fj_ir

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",0.34,0.25,0.41,0.00,1.0
"[550, 600)",0.32,0.37,0.00,0.32,1.0
"[600, 650)",0.00,0.11,0.25,0.64,1.0
Huevo,0.25,0.25,0.23,0.27,1.0


### c) *Media, mediana y moda del peso de los huevos*

En primer lugar, se crea un vector con las **marcas de clase** de $Y$, $y_{j}$:

In [11]:
l1 = 25 #Límite inferior del primer intervalo
l2 = 27 #Límite superior del primer intervalo
a = l2 - l1 #Amplitud de los intervalos
s = 4 #Número de intervalos

y1 = (l1 + l2)/2 #Primera marca de clase

mc = it.count(start = y1, step = a)
yj = [next(mc) for elemento in range(s)]
yj

[26.0, 28.0, 30.0, 32.0]

Y a partir de él, las frecuencias absolutas marginales de $Y$ y el número total de observaciones, se determina la **media** ($\bar{y}$):

In [12]:
media = round(((yj * n·j).sum())/n, 2) 
media

29.05

Para determinar la **mediana** ($Me_{y}$), en primer lugar se determina el intervalo de $Y$ cuya **frecuencia relativa acumulada marginal** ($F_{.j}$) es superior a 0,5:

In [13]:
F·j = np.cumsum(n·j/n)

intervalos = []

for intervalo, frecuencia in F·j.items():
    if frecuencia > 0.5:
        intervalos.append(intervalo)
        
int_med = intervalos[0] #La mediana se encuentra en el inmediatamente superior
int_med

'[29, 31)'

Se toma el extremo inferior de este intervalo:

In [14]:
l_med = int(int_med[1:3])
l_med

29

Se determina la **frecuencia absoluta acumulada marginal** ($N_{.j}$) del intervalo anterior:

In [15]:
for intervalo, frecuencia in F·j.items():
    if frecuencia <= 0.5:
        intervalos.append(intervalo)
        
int_ant = intervalos[-1] #Intervalo inmediatamente inferior al que contiene la mediana
int_ant

N·j = np.cumsum(n·j)

N·j_ant = N·j[int_ant]
N·j_ant

55

Y la frecuencia absoluta marginal del intervalo que contiene la mediana:

In [16]:
n·j_med = n·j[int_med]
n·j_med

25

Finalmente, la mediana se calcula como:

In [17]:
mediana = int(l_med + a * ((n/2 - N·j_ant)/n·j_med))
mediana

29

Con respecto a la **moda**, se puede estimar el intervalo modal, aquel con mayor **frecuencia relativa marginal** ($f_{.j}$):

In [18]:
f·j = n·j/n

for intervalo, frecuencia in f·j.items():
    if frecuencia == f·j.max():
        print(intervalo)

[31, 33)


### d) *Nivel de representatividad de la media del peso de la madre cuando el huevo está comprendido entre 25 y 27*

En primer lugar, se determinan las **marcas de clase**, pero en este caso para la variable $X$, el peso de la madre ($x_{i}$):

In [19]:
l1 = 500 #Límite inferior del primer intervalo
l2 = 550 #Límite superior del primer intervalo
a = l2 - l1 #Amplitud de los intervalos
s = 3 #Número de intervalos

x1 = (l1 + l2)/2 #Primera marca de clase

mc = it.count(start = x1, step = a)
xi = [next(mc) for elemento in range(s)]
xi

[525.0, 575.0, 625.0]

Resulta interesante recodificar la tabla de frecuencias bivariada usando las marcas de clase en vez de los intervalos:

In [20]:
tabla_2d_mc = pd.crosstab(index = x, columns = y, margins = True)

tabla_2d_mc.index = xi + ["Huevo"]
tabla_2d_mc.columns = yj  + ["Madre"]

tabla_2d_mc

,26.0,28.0,30.0,32.0,Madre
525.0,15,11,18,0,44
575.0,12,14,0,12,38
625.0,0,3,7,18,28
Huevo,27,28,25,30,110


A partir de las marcas de clase y de las **frecuencias relativas condicionadas** $f_{i|1}$, se puede determinar la **media** ($\bar{x}$) de $X$ en relación al primer intervalo de $Y$:

In [21]:
media = sum(xi * fi_1)

media_r = round(media, 2)
media_r

547.22

Y a partir de la media se puede calcular la **desviación típica** ($S_{x}$):

In [22]:
varianza = sum(np.power(xi, 2) * fi_1) - media**2

desv_tip = mt.sqrt(varianza)

desv_tip_r = round(desv_tip, 2)
desv_tip_r

24.85

Con ambas, se calcula el **coeficiente de variación** ($CV$), que es el que indica el nivel de representatividad de la media:

In [23]:
cv = round(desv_tip/media, 3)
cv

0.045

### e) *Independencia de las variables*

Se va a estimar a partir del par $(x_{1}, y_{1})$:

In [43]:
f11 = tabla_2d.iloc[0, 0]/n
f1· = ni·.iloc[0]/n
f·1 = n·j.iloc[0]/n

f11 == f1· * f·1

False

Dado que no se verifica la igualdad, las variables no son independientes.

### f) *Dependencia lineal de las variables*

Se va a estimar a través del **coeficiente de correlación de Pearson** ($r$); en primer lugar, se calculan las varianzas de ambas variables:

In [46]:
fi· = ni·/n #Frecuencias relativas marginales de x

media_x = sum(xi * fi·)

var_x = sum(np.power(xi, 2) * fi·) - media_x**2

var_x_r = round(var_x, 2)
var_x_r

1583.47

In [47]:
f·j = n·j/n #Frecuencias relativas marginales de y

media_y = sum(yj * f·j)

var_y = sum(np.power(yj, 2) * f·j) - media_y**2

var_y_r = round(var_y, 2)
var_y_r

5.14

En segundo lugar, la **covarianza** ($S_{XY}$):

In [76]:
a = (yj * np.array([xi[0]] * 4) * tabla_2d.iloc[0, ])/n  
b = (yj * np.array([xi[1]] * 4) * tabla_2d.iloc[1, ])/n 
c = (yj * np.array([xi[2]] * 4) * tabla_2d.iloc[2, ])/n
 
covarianza = (a.sum() + b.sum() + c.sum()) - (media_x * media_y)

covarianza_r = round(covarianza, 2)
covarianza_r

44.03

Y, finalmente, el coeficiente:

In [78]:
r = round(covarianza/mt.sqrt(var_x * var_y), 2)
r

0.49